#### 1) Downloaded the PDC 1, PDC 5, PDC 6 Gene of Saccharomyces cerevisiae

In [5]:
# Downloading the PDC Gene's (1,5,6)
from Bio import SeqIO, Entrez

Entrez.email = "sameer22439@iiitd.ac.in"

# Function to download gene sequence from NCBI
def download_gene_sequence(output_file, accession_number, start_pos, end_pos, reverse_complement=False):
    
    # Fetches the gene sequence from NCBI using accession number, start, and end positions. Optionally takes the reverse complement if specified.
    handle = Entrez.efetch(db="nucleotide", id=accession_number, rettype="fasta", seq_start=start_pos, seq_stop=end_pos)

    gene_record = SeqIO.read(handle, "fasta")

    handle.close()

    if reverse_complement: # If reverse complement is specified, reverse complement the sequence
        gene_record.seq = gene_record.seq.reverse_complement()

    with open(output_file, "w") as file:
        SeqIO.write(gene_record, file, "fasta")

    return gene_record

# PDC 1 requires reverse complement 
accession_number_pdc1 = "NC_001144.5" # Accession number of PDC1 gene
start_position_pdc1 = 232390 # Start position of PDC1 gene
end_position_pdc1 = 234081 # End position of PDC1 gene

print("Downloading PDC1 gene sequence")
gene_record_pdc1 = download_gene_sequence("PDC1.fsa", accession_number_pdc1, start_position_pdc1, end_position_pdc1, True)
print("Downloaded PDC1 gene sequence successfully")

# PDC 5 doesn't requires reverse complement
accession_number_pdc5 = "NC_001144.5" # Accession number of PDC5 gene
start_position_pdc5 = 410723 # Start position of PDC5 gene
end_position_pdc5 = 412414 # End position of PDC5 gene

print("Downloading PDC5 gene sequence")
gene_record_pdc5 = download_gene_sequence("PDC5.fsa", accession_number_pdc5, start_position_pdc5, end_position_pdc5)
print("Downloaded PDC5 gene sequence successfully")

# PDC 6 requires reverse complement
accession_number_pdc6 = "NC_001139.9" # Accession number of PDC6 gene
start_position_pdc6 = 651290 # Start position of PDC6 gene
end_position_pdc6 = 652981 # End position of PDC6 gene

print("Downloading PDC6 gene sequence")
gene_record_pdc6 = download_gene_sequence("PDC6.fsa", accession_number_pdc6, start_position_pdc6, end_position_pdc6, True)
print("Downloaded PDC6 gene sequence successfully")

# print(gene_record_pdc1.seq)
# print(gene_record_pdc5)
# print(gene_record_pdc6)


Downloaded PDC1 gene sequence successfully
Downloaded PDC5 gene sequence successfully
Downloaded PDC6 gene sequence successfully


#### 2) Performing Blast to compare PDC Gene Sequence across different Saccharomyces cerevisiae strains

In [6]:
from Bio.Blast import NCBIWWW, NCBIXML
from Bio import SeqIO, Entrez
from Bio.Seq import Seq

# Function to remove gaps and ambigious bases from the sequence
def removing_gaps_and_ambigious_bases(sequence):
    # Removes gaps and ambigious bases from the sequence
    final_sequence = str(sequence).replace("-", "").replace("N", "")

    while len(final_sequence) % 3 != 0:
        final_sequence = final_sequence + "N"
    
    return Seq(final_sequence)

# Function to download BLAST results for the given sequence
def download_blast_file(sequence, output_file):
    # Removes gaps and ambigious bases from the sequence
    sequence = removing_gaps_and_ambigious_bases(sequence)

    # BLAST the sequence against the NCBI database
    print("BLASTing the sequence against the NCBI database")
    result_handle = NCBIWWW.qblast("blastn", "nr", sequence, entrez_query='txid4932[Organism]')
    print("BLASTing completed successfully")
    
    with open(output_file, "w") as file:
        file.write(result_handle.read())
    
    print("BLAST results saved to", output_file)

    result_handle.close()

# Function to parse the BLAST results and save the scores to a text file
def parse_blast_results(blast_file, output_file):
    with open(blast_file, "r") as result_handle, open(output_file, "w") as output_text:
        blast_records = NCBIXML.parse(result_handle)

        for blast_record in blast_records:
            for alignment in blast_record.alignments:
                output_text.write(f"Alignment title: {alignment.title}\n\n")
                for hsp in alignment.hsps:
                    output_text.write(f"Score:   {hsp.score}\n")
                    output_text.write(f"E-value: {hsp.expect}\n")
                    output_text.write(f"Query:   {hsp.query}\n")
                    output_text.write(f"Match:   {hsp.match}\n")
                    subject_seq = str(hsp.sbjct).replace("-", "").replace("N", "")
                    output_text.write(f"Subject: {subject_seq}\n")
                    
                output_text.write("-------------------------------------------------------------\n\n")
            


# Download the BLAST results for PDC1 gene
print("Downloading BLAST file for PDC1 gene")
download_blast_file(gene_record_pdc1.seq, "PDC1_blast.xml")
print("Downloaded BLAST file for PDC1 gene successfully")
parse_blast_results("PDC1_blast.xml", "PDC1_blast_score.txt")
print("Parsed BLAST results for PDC1 gene successfully")
print("---------------------------------------------")

# Download the BLAST results for PDC5 gene
print("Downloading BLAST file for PDC5 gene")
download_blast_file(gene_record_pdc5.seq, "PDC5_blast.xml")
print("Downloaded BLAST file for PDC5 gene successfully")
parse_blast_results("PDC5_blast.xml", "PDC5_blast_score.txt")
print("Parsed BLAST results for PDC5 gene successfully")
print("---------------------------------------------")

# Download the BLAST results for PDC6 gene
print("Downloading BLAST file for PDC6 gene")
download_blast_file(gene_record_pdc6.seq, "PDC6_blast.xml")
print("Downloaded BLAST file for PDC6 gene successfully")
parse_blast_results("PDC6_blast.xml", "PDC6_blast_score.txt")
print("Parsed BLAST results for PDC6 gene successfully")
print("---------------------------------------------")

BLASTing the sequence against the NCBI database
BLASTing completed successfully
BLAST results saved to PDC1_blast.xml
Downloaded BLAST file for PDC1 gene successfully
Parsed BLAST results for PDC1 gene successfully
---------------------------------------------
BLASTing the sequence against the NCBI database
BLASTing completed successfully
BLAST results saved to PDC5_blast.xml
Downloaded BLAST file for PDC5 gene successfully
Parsed BLAST results for PDC5 gene successfully
---------------------------------------------
BLASTing the sequence against the NCBI database
BLASTing completed successfully
BLAST results saved to PDC6_blast.xml
Downloaded BLAST file for PDC6 gene successfully
Parsed BLAST results for PDC6 gene successfully
---------------------------------------------


In [7]:
from Bio.Blast import NCBIWWW, NCBIXML
from Bio import SeqIO, Entrez
from Bio.Seq import Seq

# Function to find SNPs in the BLAST results
def finding_SNP_in_blast_results(blast_file, output_file):
    with open(blast_file, "r") as result_handle, open(output_file, "w") as output_text:
        blast_records = NCBIXML.parse(result_handle)

        for blast_record in blast_records:
            for alignment in blast_record.alignments:
                output_text.write(f"Alignment title: {alignment.title}\n\n")
                for hsp in alignment.hsps:
                    output_text.write(f"Query:   {hsp.query}\n")
                    output_text.write(f"Match:   {hsp.match}\n")
                    subject_seq = str(hsp.sbjct).replace("-", "").replace("N", "")
                    output_text.write(f"Subject: {subject_seq}\n")

                    flag = 1

                    for i in range(len(hsp.query)):
                        if (hsp.query[i] != hsp.sbjct[i]):
                            if (hsp.query[i-1] == hsp.sbjct[i-1] and hsp.query[i+1] == hsp.sbjct[i+1]):
                                output_text.write(f"SNP found at position {i+1}: {hsp.query[i]} ---> {hsp.sbjct[i]}\n")
                                flag = 0
                
                    if flag == 1:
                        output_text.write("No SNP found\n")


                output_text.write("-------------------------------------------------------------\n")

                            
print("Finding SNPs in PDC1 gene")
finding_SNP_in_blast_results("PDC1_blast.xml", "PDC1_SNP.txt")
print("Finding SNPs in PDC5 gene")
finding_SNP_in_blast_results("PDC5_blast.xml", "PDC5_SNP.txt")
print("Finding SNPs in PDC6 gene")
finding_SNP_in_blast_results("PDC6_blast.xml", "PDC6_SNP.txt")

Finding SNPs in PDC1 gene
Finding SNPs in PDC5 gene
Finding SNPs in PDC6 gene


#### 3) Translation of PDC genes and their SNP containing genes

In [13]:
from Bio.Blast import NCBIWWW, NCBIXML
from Bio import SeqIO, Entrez
from Bio.Seq import Seq

# Function to translate the strains and query in the BLAST results
def translating_the_strains_and_query_in_blast_results(blast_file, output_file):
    with open(blast_file, "r") as result_handle, open(output_file, "w") as output_text:
        blast_records = NCBIXML.parse(result_handle)
        gene_sequence = None
        for blast_record in blast_records:
            for alignment in blast_record.alignments:
                output_text.write(f"Alignment title: {alignment.title}\n\n")
                for hsp in alignment.hsps:
                    gene_sequence = hsp.query
                    subject_seq = str(hsp.sbjct).replace("-", "").replace("N", "")
                    protein_seq = Seq(subject_seq).translate()
                    output_text.write(f"Subject : {subject_seq}\n")
                    output_text.write(f"Protein : {protein_seq}\n\n")

                output_text.write("-------------------------------------------------------------\n")
        
        protein_sequence_of_query = str(Seq(gene_sequence).translate())
        output_text.write("Query :   " + str(gene_sequence) + "\n")
        output_text.write("Protein : " + protein_sequence_of_query)


print("Translating PDC1 gene and it\'s strains")
translating_the_strains_and_query_in_blast_results("PDC1_blast.xml", "PDC1_protein.txt")
print("Translating PDC5 gene and it\'s strains")
translating_the_strains_and_query_in_blast_results("PDC5_blast.xml", "PDC5_protein.txt")
print("Translating PDC6 gene and it\'s strains")
translating_the_strains_and_query_in_blast_results("PDC6_blast.xml", "PDC6_protein.txt")

Translating PDC1 gene and it's strains
Translating PDC5 gene and it's strains
Translating PDC6 gene and it's strains


#### 4) Done this question using online software (http://genetics.bwh.harvard.edu/pph2/index.shtml) 

##### Links given below are the SNP's in the Query Protein Sequence and it's strains returned during BLASTN process for 3 different genes of Saccharomyces cerevisiae, i.e. PDC 1, PDC 5 and PDC 6


session id : 0ee7cc0802f7d10ef08cc68d35b7ead9646c9924

link 1 : http://genetics.bwh.harvard.edu/ggi/pph2/0ee7cc0802f7d10ef08cc68d35b7ead9646c9924/9644046.html : PDC 1

link 2 : http://genetics.bwh.harvard.edu/ggi/pph2/0ee7cc0802f7d10ef08cc68d35b7ead9646c9924/9644047.html : PDC 1

link 3 : http://genetics.bwh.harvard.edu/ggi/pph2/0ee7cc0802f7d10ef08cc68d35b7ead9646c9924/9644048.html : PDC 1

link 4 : http://genetics.bwh.harvard.edu/ggi/pph2/0ee7cc0802f7d10ef08cc68d35b7ead9646c9924/9644049.html : PDC 1

link 5 : http://genetics.bwh.harvard.edu/ggi/pph2/0ee7cc0802f7d10ef08cc68d35b7ead9646c9924/9644050.html : PDC 1

link 6 : http://genetics.bwh.harvard.edu/ggi/pph2/0ee7cc0802f7d10ef08cc68d35b7ead9646c9924/9644054.html : PDC 5

link 7 : http://genetics.bwh.harvard.edu/ggi/pph2/0ee7cc0802f7d10ef08cc68d35b7ead9646c9924/9644057.html : PDC 5

link 8 : http://genetics.bwh.harvard.edu/ggi/pph2/0ee7cc0802f7d10ef08cc68d35b7ead9646c9924/9644058.html : PDC 5

link 9 : http://genetics.bwh.harvard.edu/ggi/pph2/0ee7cc0802f7d10ef08cc68d35b7ead9646c9924/9644060.html : PDC 5

link 10 : http://genetics.bwh.harvard.edu/ggi/pph2/0ee7cc0802f7d10ef08cc68d35b7ead9646c9924/9644061.html : PDC 5

link 11 : http://genetics.bwh.harvard.edu/ggi/pph2/0ee7cc0802f7d10ef08cc68d35b7ead9646c9924/9645333.html : PDC 6

link 12 : http://genetics.bwh.harvard.edu/ggi/pph2/0ee7cc0802f7d10ef08cc68d35b7ead9646c9924/9645338.html : PDC 6

link 13 : http://genetics.bwh.harvard.edu/ggi/pph2/0ee7cc0802f7d10ef08cc68d35b7ead9646c9924/9645353.html : PDC 6

link 14 : http://genetics.bwh.harvard.edu/ggi/pph2/0ee7cc0802f7d10ef08cc68d35b7ead9646c9924/9645357.html : PDC 6

link 15 : http://genetics.bwh.harvard.edu/ggi/pph2/0ee7cc0802f7d10ef08cc68d35b7ead9646c9924/9645373.html : PDC 6